In [ ]:
import pandas as pd
import numpy as np
import threading
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import requests
import astral
from astral.sun import sun
from uszipcode import SearchEngine, Zipcode
from sklearn.linear_model import LinearRegression

Data from https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95
<br/>Long term: Add weather data

In [ ]:
df = pd.read_csv(r'../data/Motor_Vehicle_Collisions_-_Crashes.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

### Data cleaning

##### Cleaning notes

1. `ZIP` to `int`
2. `CRASH DATE` and `CRASH TIME` to `datetime`
3. Remove outlying latitude/longitude data
4. Get categorical count totals
5. Compare `NUMBER OF PERSONS INJURED` to other columns' totals
6. Add day/night column
7. Add season column
8. Add rush hour column
9. Clear up missing ZIP data

##### Compare `NUMBER OF PERSONS INJURED`

In [ ]:
injured_audit = df['NUMBER OF PEDESTRIANS INJURED'] + df['NUMBER OF CYCLIST INJURED'] + df['NUMBER OF MOTORIST INJURED'] == df['NUMBER OF PERSONS INJURED']
injured_audit.value_counts()

##### Remove outlying latitude/longitude data

From the [New York Times](https://www.nytimes.com/2001/07/01/nyregion/fyi-027065.html#:~:text=Vincent%20area%20of%20the%20Bronx,degrees%2015%20minutes%20west%20longitude.):

According to ''A Natural History of New York City'' by John Kieran (Houghton Mifflin, 1959), the easternmost point is in Glen Oaks, Queens, at about 73 degrees 42 minutes west longitude, and the extreme northern point is the bank of the Hudson in the Mount St. Vincent area of the Bronx, at about 40 degrees 55 minutes north latitude. The southernmost and westernmost points almost coincide in Tottenville, Staten Island, at 40 degrees 30 minutes north latitude and 74 degrees 15 minutes west longitude.

In [ ]:
df = df[df['LATITUDE'] >= 40]
df = df[df['LATITUDE'] <= 41]

In [ ]:
df = df[df['LONGITUDE'] <= -73.3]
df = df[df['LONGITUDE'] >= -74.2]

In [ ]:
boroughs = tuple(df['BOROUGH'].dropna().unique())
colors = ['r','g','b','c','y']

for i, borough in enumerate(boroughs):
    plot_borough = df[df['BOROUGH'] == borough]
#     plt.subplot(111 + boroughs.index(borough)*10)    
    _ = plt.plot(plot_borough['LONGITUDE'], plot_borough['LATITUDE'], colors[i]+'.', alpha=0.005)
    _ = plt.xlim(-74.2,-73.8)
    _ = plt.ylim(40.5,40.9)
# plt.imshow(extent=[-74,-73,40,41])
plt.show()

##### Datetime conversion

In [ ]:
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'])

# df['CRASH TIME'] = (df['CRASH DATE'] + ' ' + df['CRASH TIME']).apply(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M'))

In [ ]:
df[['CRASH DATE','CRASH TIME']].info()

##### ZIP replacement

In [ ]:
np.sort(df['ZIP CODE'].astype('str').unique())[:10]

In [ ]:
empty = np.sort(df['ZIP CODE'].astype('str').unique())[0]
empty_cells = df['ZIP CODE'].isin([empty, np.nan])
df['EMPTY'] = empty_cells

In [ ]:
len(empty_cells[empty_cells==True])

In [ ]:
# build zip code search function
search = SearchEngine(simple_zipcode=True)
def get_zipcode(lat,lng):
    result = search.by_coordinates(lat = lat, lng = lng, returns = 1)
    return int(result[0].zipcode)

In [ ]:
for i,j in df.loc[df['EMPTY'] == True, ['LATITUDE', 'LONGITUDE']].drop_duplicates().values:
    df.loc[(df['LATITUDE'] == i) & (df['LONGITUDE'] == j), 'ZIP CODE'] = get_zipcode(i,j)

In [ ]:
empty = np.sort(df['ZIP CODE'].astype('str').unique())[0]
empty_cells = df['ZIP CODE'].isin([empty, np.nan])
len(empty_cells[empty_cells==True])

##### Fill borough data

In [ ]:
len(df[df['BOROUGH'].isnull()])

In [ ]:
manhattan_zips = [
    10026,10027,10030,10037,10039, # Harlem
    10001,10011,10018,10019,10020,10036, # Chelsea/Clinton
    10029,10035, #East Harlem
    10010,10016, 10017, 10022, # Grammercy Park/Murray Hill
    10012, 10013, 10014, # Greenwich Village/SoHo
    10004, 10005, 10006, 10007, 10038, 10280, # Lower Manhattan
    110002, 10003, 10009, # Lower East Side
    10021, 10028, 10044, 10065, 10075, 10128, # Upper East Side
    10023, 10024, 10025, # Upper West Side
    10031, 10032, 10033, 10034, 10040 # Inwood/Washington Heights
]

brooklyn_zips = [
    11212, 11213, 11216, 11233, 11238, # Central Brooklyn
    11209, 11214, 11228, # Southwest Brooklyn
    11204, 11218, 11219, 11230, # Borough Park
    11234, 11236, 11239, # Canarsie/Flatlands
    11223, 11224, 11229, 11235, # Southern Brooklyn
    11201, 11205, 11215, 11217, 11231, # Northwest Brooklyn
    11203, 11210, 11225, 11226, # Flatbush
    11207, 11208, # East New York/New Lots
    11211, 11222, # Greenpoint
    11220, 11232, # Sunset Park
    11206, 11221, 11237 # Bushwick/Williamsburg
]

queens_zips = [
    11361, 11362, 11363, 11364, # Northeast Queens
    11354, 11355, 11356, 11357, 11358, 11359, 11360, # North Queens
    11365, 11366, 11367, # Central Queens
    11412, 11423, 11432, 11433, 11434, 11435, 11436, # Jamaica
    11101, 11102, 11103, 11104, 11105, 11106, # Northwest Queens
    11374, 11375, 11379, 11385, # West Central Queens
    11691, 11692, 11693, 11694, 11695, 11697, # Rockaways
    11004, 11005, 11411, 11413, 11422, 11426, 11427, 11428, 11429, # Southeast Queens
    11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421, # Southwest Queens
    11368, 11369, 11370, 11372, 11373, 11377, 11378 # West Queens
]

si_zips = [
    10302, 10303, 10310, # Port Richmond
    10306, 10307, 10308, 10309, 10312, # South Shore
    10301, 10304, 10305, # Stapleton/St. George
    10314 # Mid-Island
]

bronx_zips = [
    10453, 10457, 10460, # Central Bronx
    10458, 10467, 10468, # Bronx Park/Fordham
    10451, 10452, 10456, # High Bridge/Morrisania
    10454, 10455, 10459, 10474, # Hunts Point/Mott Haven
    10463, 10471, # Kingsbridge/Riverdale
    10466, 10469, 10470, 10475, # Northeast Bronx
    10461, 10462,10464, 10465, 10472, 10473 # Southeast Bronx
]

In [ ]:
boroughs = ['MANHATTAN','BROOKLYN','QUEENS','STATEN ISLAND','BRONX']
zips = [manhattan_zips, brooklyn_zips, queens_zips, si_zips, bronx_zips]

for borough, zip_code in zip(boroughs, zips):
    df.loc[df['ZIP CODE'].isin(zip_code), 'BOROUGH'] = borough

In [ ]:
len(df[df['BOROUGH'].isnull()])

##### Add columns

In [ ]:
# add RUSH HOUR column for hours between 6-10 AM and 4-8 PM
df['RUSH HOUR'] = 0

morning_rush = (df['CRASH TIME'].dt.time > dt.time(5,0)) & (df['CRASH TIME'].dt.time < dt.time(10,0))
afternoon_rush = (df['CRASH TIME'].dt.time > dt.time(16,0)) & (df['CRASH TIME'].dt.time < dt.time(20,0))

df.loc[(morning_rush == True) | (afternoon_rush == True), 'RUSH HOUR'] = 1

In [ ]:
# add SEASON column based on meteorological definition
df.loc[df['CRASH DATE'].dt.month.isin([3,4,5]) == True, 'SEASON'] = 'spring'
df.loc[df['CRASH DATE'].dt.month.isin([6,7,8]) == True, 'SEASON'] = 'summer'
df.loc[df['CRASH DATE'].dt.month.isin([9,10,11]) == True, 'SEASON'] = 'fall'
df.loc[df['CRASH DATE'].dt.month.isin([12,1,2]) == True, 'SEASON'] = 'winter'

In [ ]:
df[['CRASH DATE','RUSH HOUR','SEASON']].sample(5)

In [ ]:
location = astral.LocationInfo(name='New York', region='New York', timezone='America/New_York', latitude=40.7, longitude=-74)

def get_sunrise_sunset(sun_action, time_zone, crash_date):
    time = sun(location.observer, tzinfo=time_zone, date=crash_date.date())[sun_action].time()
    return time

In [ ]:
df['SUNRISE'] = df.apply(lambda x: get_sunrise_sunset(sun_action='sunrise',\
                                                      time_zone='America/New_York',\
                                                      crash_date=x['CRASH DATE']), axis=1)
df['SUNSET'] = df.apply(lambda x: get_sunrise_sunset(sun_action='sunset',\
                                                     time_zone='America/New_York',\
                                                     crash_date=x['CRASH DATE']), axis=1)

In [ ]:
df['DURING DAYTIME'] = 0
after_sunrise = (df['CRASH TIME'].dt.time > df['SUNRISE']) & (df['CRASH TIME'].dt.time < df['SUNSET'])
df.loc[after_sunrise == True, 'DURING DAYTIME'] = 1

In [ ]:
df[['CRASH TIME','SUNRISE','SUNSET','DURING DAYTIME']].sample(5)

In [ ]:
# add TOTAL PEDESTRIAN CASUALTIES target variable
df['TOTAL PEDESTRIAN CASUALTIES'] = df['NUMBER OF PEDESTRIANS INJURED'] + df['NUMBER OF PEDESTRIANS KILLED'] + df['NUMBER OF CYCLIST INJURED'] + df['NUMBER OF CYCLIST KILLED']

In [ ]:
df.info()

In [ ]:
# add weekdays
weekdays = ['SUNDAY','MONDAY','TUESDAY','WEDNESDAY','THURSDAY','FRIDAY','SATURDAY']
df['WEEKDAY'] = df['CRASH DATE'].apply(lambda x: weekdays[x.weekday()])
df['WEEKDAY'].sample(10)

In [ ]:
# add weekend dummy
df['WEEKEND'] = 0
weekend = df['WEEKDAY'].isin(['SATURDAY','SUNDAY'])
df.loc[weekend == True, 'WEEKEND'] = 1

##### Add holiday distance data

In [ ]:
# df = pd.read_csv(r'data/clean_df.csv.gz', index_col=0)
# df['CRASH DATE'] = df['CRASH DATE'].astype('datetime64')
# df['CRASH TIME'] = df['CRASH TIME'].astype('datetime64')
# df.head()

In [ ]:
df.info()

In [ ]:
christmases = df['CRASH DATE'].apply(lambda x: dt.datetime(x.year,12,25))
new_years = df['CRASH DATE'].apply(lambda x: dt.datetime(x.year,1,1))
independence_days = df['CRASH DATE'].apply(lambda x: dt.datetime(x.year,7,4))
christmases.sample(5)

In [ ]:
df['DAYS FROM NEW YEARS'] = df['CRASH DATE'] - new_years
df['DAYS FROM CHRISTMAS'] = df['CRASH DATE'] - christmases
df['DAYS FROM JULY 4'] = df['CRASH DATE'] - independence_days

In [ ]:
df.sample(5)

### Factor clean-up

In [ ]:
with pd.option_context('display.max_rows', None):
    print(df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().sort_values(ascending=False))

In [ ]:
# Remove all factors that cannot be prevented by a cop
preventables = [
    'Unspecified',
    'Driver Inattention/Distraction',
    'Failure to Yield Right-of-Way',
    'Following Too Closely',
    'Backing Unsafely',
    'Other Vehicular',
    'Passing or Lane Usage Improper',
    'Turning Improperly',
    'Passing Too Closely',
    'Unsafe Lane Changing',
    'Traffic Control Disregarded',
    'Driver Inexperience',
    'Alcohol Involvement',
    'Unsafe Speed',
    'Reaction to Uninvolved Vehicle',
    'Prescription Medication',
    'Outside Car Distraction',
    'Oversized Vehicle',
    'View Obstructed/Limited',
    'Passenger Distraction',
    'Aggressive Driving/Road Rage',
    'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
    'Failure to Keep Right',
    'Other Electronic Device',
    'Reaction to Other Uninvolved Vehicle',
    'Animals Action',
    'Lane Marking Improper/Inadequate',
    'Traffic Control Device Improper/Non-Working',
    'Drugs (illegal)',
    'Drugs (Illegal)',
    'Cell Phone (hand-Held)',
    'Cell Phone (hands-free)',
    'Other Lighting Defects',
    'Using On Board Navigation Device',
    'Headlights Defective',
    'Eating or Drinking',
    'Tinted Windows',
    'Cell Phone (hand-held)',
    'Windshield Inadequate',
    'Texting',
    'Listening/Using Headphones'
]

df = df[df['CONTRIBUTING FACTOR VEHICLE 1'].isin(preventables)]
df.shape

### Street name clean

In [ ]:
df = pd.read_csv(r'../data/clean_df.csv.gz')

In [ ]:
alt_terms = ['STREET',
             'AVENUE',
             'BOULEVARD',
             'ROAD',
             'DRIVE',
             'PARKWAY',
             'EXPRESSWAY'
            ]
clean_terms = ['ST',
               'AVE',
               'BLVD',
               'RD',
               'DR',
               'PKWY',
               'EXPWY'
              ]
def alt_terms_clean(string):
    for alt, clean in zip(alt_terms, clean_terms):
        string = string.strip().upper().replace(alt, clean)
    return string

In [ ]:
df['ON STREET NAME'].fillna('', inplace=True)
df['ON STREET NAME'] = df['ON STREET NAME'].map(alt_terms_clean)

df['CROSS STREET NAME'].fillna('', inplace=True)
df['CROSS STREET NAME'] = df['CROSS STREET NAME'].map(alt_terms_clean)

In [ ]:
df.to_csv(r'../data/clean_df.csv.gz')